    Ontology: Contains all objects
    Subclass: Contains only clases in subclass relations
    exp: Explanations of subclasses in ontologies
    (For each subclass there is a exp)

        For each chosen_explanation in exp:
            List_of_candidate_objects: Objects that are IN chosen_explanation AND that are NOT IN subclass relation
            chosen_candidate = func(List_of_candidate_objects)
            signature=chosen_candidate
            chosen_explanation_ontology_1 = forget(chosen_explanation,signature)
            generate_subclasses(chosen_explanation_ontology_1)
            #Possible for one iteration for each explanation

In [5]:
import os
from IPython.display import clear_output

In [6]:
from shutil import move

In [7]:
from subprocess import Popen, PIPE, CalledProcessError
from shutil import copyfile
from os import listdir
from os.path import isfile, join
import re

In [8]:
def move_file(src,dest):
    copyfile(src,dest)
    os.remove(src)

def get_path( path):
    path = os.path.join(*path)
    return(path)

def get_all_files(mypath, _filter='exp-'):
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    return ([f for f in onlyfiles if 'exp-' in f])

def get_name(name, prefix='sc', suffix = '.nt', iteration = 0):
    names= name.split('-')
    if len(names) == 1:
        return(f'{prefix}-{name}-{iteration}{suffix}')
    else:
        i = int(names[-1])
        return(f'{prefix}-{names[1]}-{i}{suffix}')
    
def get_subclasses(path, input_file):
    input_ontology = get_path([path,input_file])
    subclass_file = 'subClasses.nt'
    subclass_file_path = get_path([path,subclass_file])
    cmd = 'java -jar kr_functions.jar ' + 'saveAllSubClasses' + " " + input_ontology
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
        subclass_file_path = get_path([path,subclass_file])
        new_subclass_file = get_name(input_file.strip('.owl'))
        new_subclass_path = get_path([path,new_subclass_file])
        move_file(subclass_file_path,new_subclass_path)
        print(f'Subclass file for {input_file} stored at {new_subclass_path}')
        return(new_subclass_path)
    
def get_explanations(path, input_file, subclass_file):
    input_ontology = get_path([path,input_file])
    input_subclass = get_path([path,subclass_file])
    cmd = 'java -jar kr_functions.jar ' + 'saveAllExplanations' + " " + input_ontology + " " + input_subclass
    exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
    new_full_path = get_path([path,exp_folder])
    os.mkdir(new_full_path)
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
#         exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
#         new_full_path = get_path([path,exp_folder])
#         os.mkdir(new_full_path)
        exp_files = get_all_files(path,_filter = 'exp-')
        for file in exp_files:
            new_path = get_path([path, exp_folder, file])
            old_path = get_path([path,file])
            move_file(old_path,new_path)
        print(f'Explanation files for {input_file} stored at {new_full_path}')
        return(new_full_path)

In [9]:
def forget_from_exp(exp_path, exp_candidates, func = 'min'):

    candidate_choosing = func_dict[func]
    exp_file = exp_path.split('\\')[-1]
    new_file = get_name(exp_file, prefix = f'{func}_signatures', suffix='')
    new_path = get_path([exp_path.strip(exp_file)[:-1],new_file])


    new_ontology_name = get_name(exp_file, prefix=f'{func}_ont', suffix='')
    new_ontology_path = get_path([exp_path.strip(exp_file)[:-1],new_ontology_name])
    

    
    new_subclass_name = get_name(exp_file, prefix=f'{func}_sc', suffix='')
    new_subclass_path = get_path([exp_path.strip(exp_file)[:-1],new_subclass_name])

    os.mkdir(new_path)
    os.mkdir(new_ontology_path)
    os.mkdir(new_subclass_path)
    for exp,candidates in exp_candidates.items():
        chosen = None
#         chosen = candidate_choosing(candidates)

#         signature = create_signature_exp([chosen],new_path, exp)
#         forgotten = forget_exp(exp_path, exp,signature)
#         new_forgotten = get_path([new_ontology_path,forgotten.split('\\')[-1]])
#         move_file(forgotten,new_forgotten)
#         subclass=get_subclasses(new_ontology_path, new_forgotten.split('\\')[-1])
#         new_subclass = get_path([new_subclass_path, subclass.split('\\')[-1]])
#         move_file(subclass,new_subclass)
#         print(new_subclass,subclass)
#         print(new_forgotten)
#         print(signature)
#         break

        try:
            chosen = candidate_choosing(candidates)

            signature = create_signature_exp([chosen],new_path, exp)
            forgotten = forget_exp(exp_path, exp,signature)
            new_forgotten = get_path([new_ontology_path,forgotten.split('\\')[-1]])
            move_file(forgotten,new_forgotten)
            subclass=get_subclasses(new_ontology_path, new_forgotten.split('\\')[-1])
            new_subclass = get_path([new_subclass_path, subclass.split('\\')[-1]])
            move_file(subclass,new_subclass)
#             print(new_forgotten)
#             print(signature)
#             break
        except:
            print(exp,candidates)
    
    return(new_path, new_ontology_path,new_subclass_path)

        
def create_signature_exp(signature,path,ontology_file=None, name = None):
#     print(ontology_file, signature,path)

    if not name:
        name = get_name('-'+ontology_file.strip('.omn'), prefix = 'signature', suffix='.txt')
    
    signature_path = get_path([path,name])
    with open(signature_path, 'w') as f:
        for item in signature:
            f.write("%s\n" % item)
    return signature_path

def forget_exp(path,input_file, signature_file, method = 3, ext = '.omn'):
    method = str(method)
#     path = path.strip('\\datasets')
    input_ontology = get_path([path,input_file])
    signature = get_path([path,signature_file])
    cmd = 'java -cp lethe-standalone.jar uk.ac.man.cs.lethe.internal.application.ForgettingConsoleApplication --owlFile ' + input_ontology + ' --method ' + method  + ' --signature ' + signature
#     new_full_path = get_path([path,exp_folder])
#     os.mkdir(new_full_path)
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
#         exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
#         new_full_path = get_path([path,exp_folder])
#         os.mkdir(new_full_path)
        new_ontology = get_name('-'+input_file.strip(ext), iteration=1, prefix='ont', suffix='.owl')
        new_ontology_path = get_path([path,new_ontology])
        result_file = 'result.owl'

        move_file(result_file,new_ontology_path)
#         print(f'New ontology saved at {new_ontology_path}')
        return(new_ontology_path)    

In [10]:
def forget(path,input_file, signature_file, method = 3, ext = '.owl'):
    method = str(method)
#     path = path.strip('\\datasets')
    input_ontology = get_path([path,input_file])
    signature = get_path([path,signature_file])
    cmd = 'java -cp lethe-standalone.jar uk.ac.man.cs.lethe.internal.application.ForgettingConsoleApplication --owlFile ' + input_ontology + ' --method ' + method  + ' --signature ' + signature
#     new_full_path = get_path([path,exp_folder])
#     os.mkdir(new_full_path)
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
#         exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
#         new_full_path = get_path([path,exp_folder])
#         os.mkdir(new_full_path)
#         new_ontology = get_name(input_file.strip(ext), iteration=1, prefix='d', suffix='.owl')
#         new_ontology_path = get_path([path,new_ontology])
        result_file = 'result.owl'

#         move_file(result_file,new_ontology_path)
#         print(f'New ontology saved at {new_ontology_path}')
        return(result_file)    

In [11]:
from tqdm import tqdm

In [12]:
def get_min(_dict):
    return min(_dict, key=_dict.get)
def get_max(_dict):
    return max(_dict, key=_dict.get)
def get_random(_dict):
    import random
    return random.choice(list(_dict.keys()))
func_dict = {'min':get_min, 'max':get_max, 'rand':get_random}

In [13]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def get_classes_sc(file):
    candidates = []
    candidate_classes ={}
    non_candidates = []
    non_candidate_classes = []
    all_classes = extract_classes(file)
    return(all_classes)

def get_classes_owl(path):
    with open(path) as f:
        text = f.read()
    _ = text.split(r'Object Properties')
    result = []
    regex = r'".+#.+"'
    
    for i in tqdm(_[-1].split('>')):
        t = re.search(regex,i)
        if t:

            result.append(t.string[t.start()+1:t.end()-1])
    # result = list(re.finditer(regex,text))
    # result = [i.string[i.start():i.end()] for i in result]
    result_count = {}
    for i in tqdm(result):
        result_count[i] = result.count(i)
#     _keys = list(result_count.keys())
    # for i in remove:
    #     for k in _keys:
    #         if i in k.lower():
    #               result_count.pop(k, None)
    return(result_count)
    
    


def extract_classes(file, remove = ['subclassof']):
    with open(file) as f:
        text = f.read()
    result = []
    regex = r'<.+#.+'
    for i in tqdm(text.split('>')):
        t = re.search(regex,i)
        if t:

            result.append(t.string[t.start()+1:t.end()])
    # result = list(re.finditer(regex,text))
    # result = [i.string[i.start():i.end()] for i in result]
    result_count = {}
    for i in tqdm(result):
        result_count[i] = result.count(i)
    _keys = list(result_count.keys())
    for i in remove:
        for k in _keys:
            if i in k.lower():
                  result_count.pop(k, None)
        
    return(result_count)
    

In [14]:
def create_signature(signature,path,ontology_file=None, name=None):
    if not name:
        name = get_name(ontology_file.strip('.owl'), prefix = 'signature', suffix='.txt')
    signature_path = get_path([path,name])
    with open(signature_path, 'w') as f:
        for item in signature:
            f.write("%s\n" % item)
    return signature_path



In [15]:
def get_objects_not_in_sc(sc_path, owl_path, ontology_name):
    sc_classes = get_classes_sc(sc_path)
    objects_owl = get_classes_owl(owl_path)
    candidates = [_object for _object in  list(objects_owl.keys()) if _object not in list(sc_classes.keys())]
    candidate_count= {}
    for candidate in candidates:
        #gets count of everything that is in ontology but not in subclass relationship
        if ontology_name in candidate:
            candidate_count[candidate] = objects_owl[candidate]
    return(candidate_count)

In [16]:
def get_candidates_exp(path, not_sc_objects):
    exp_files = get_all_files(path, _filter = 'exp-')
    files = [get_path([path,i]) for i in exp_files]
    exp_candidates = {}
    for file,exp in zip(files,exp_files):
#         with open(file, 'r') as f:
#             text = f.read()
        classes = extract_classes(file)
        candidate_classes = {}
        for _class, count in classes.items():
            if _class in not_sc_objects.keys():
                candidate_classes[_class] = count
        exp_candidates[exp] = candidate_classes

    clear_output()
    return exp_candidates


In [17]:
def move_folders(signature_path, ontology_path, sc_path, func, path, input_ontology):   
    ontology_name = input_ontology.split('.')[0]
    new_path = get_path([path,f'{func}_{ontology_name}'])
    for folder in [signature_path, ontology_path, sc_path]:
        original_path = folder
        new_folder = folder.split('\\')[-1]
        
        full_path = get_path([new_path, new_folder])
        move(original_path,full_path)
    return(new_path)

def forgetting_loop(min_path, candidate_objects):
    ontology_folder = [get_path([min_path, f]) for f in listdir(min_path) if 'ont' in f][0]
    ontology_paths = [get_path([ontology_folder,f]) for f in listdir(ontology_folder) if isfile(join(ontology_folder, f))]
    signature_folder = [get_path([min_path, f]) for f in listdir(min_path) if 'signatures' in f][0]
    singature_paths = [get_path([signature_folder,f]) for f in listdir(signature_folder) if isfile(join(signature_folder, f))]
    new_ontology_paths = []
    for i,j  in zip(ontology_paths, singature_paths):
        file = i.split('\\')[-1]
        name = i.split('\\')[-1].split('.')[0]
        
        signature_file = j.split('\\')[-1]
#         signature_name = i.split('\\')[-1].split('.')[0]
        dest = get_path([min_path,name])
    #     temp = get_subclasses(ontology_folder,file)s
        try:
            os.mkdir(dest)
        except FileExistsError:
            pass
        dest_file = get_path([dest,'ont-0.owl'])
        dest_signature_file  = get_path([dest, 'signature-0.txt'])
        copyfile(i, dest_file)
        copyfile(j, dest_signature_file)
        new_ontology_paths.append(dest)
        

        
    
    for i in new_ontology_paths:
        stop = False
        iteration = 0
        while not stop:

            name = f'ont-{iteration}.owl'
            path_to_ontology = get_path([i, name])
            _temp =  get_classes_owl(path_to_ontology)
            _candidates = [c for c in _temp.keys() if c in candidate_objects.keys()]
            new_candidates = {}
            for c in _candidates:
                new_candidates[c] = _temp[c]
            print(_candidates)
    #         candidate_objects = get_objects_not_in_sc(subclass, get_path([path,input_ontology]), 'untitled-ontology-55')
            if _candidates:
                iteration+=1
                new_name = f'ont-{iteration}.owl'
                signature = func_dict[func](new_candidates)
                signature_name = 'signature-'+new_name.split('-')[1].strip('.owl')+'.txt'

                signature_path = create_signature(signature = [signature], path=i, name=signature_name)
                _new = forget(i,name,signature_name)

                print(i,signature_path)
                print('--------------')
#                 iteration+=1
#                 new_name = f'ont-{iteration}.owl'
                move(_new, get_path([i, new_name]))
            else:
                stop=True
#         stop=True
#     print(i)

    #     break

In [14]:
path = "F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets"
input_ontology = "pizza_super_simple.owl"

In [15]:

subclass=get_subclasses(path, input_ontology)

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\pizza_super_simple.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\pizza_super_simple.owl
Format : RDF/XML Syntax
--------


Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\subClasses.nt
--------


Subclass file for pizza_super_simple.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-pizza_super_simple-0.nt


In [16]:
candidate_objects = get_objects_not_in_sc(subclass, get_path([path,input_ontology]), 'pizza.owl')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 247/247 [00:00<?, ?it/s]


In [18]:
explanations = get_explanations(path, input_ontology, subclass)

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\pizza_super_simple.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\pizza_super_simple.owl
Format : RDF/XML Syntax
--------


Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-pizza_super_simple-0.nt
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-pizza_super_simple-0.nt
Format : Turtle Syntax
--------


Saving all explanations for the subsumptions located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-pizza_super_simple-0.nt



Computing explanation for: <http://www.co-ode.org/ontologies/pizza/pizza.owl#American> rdfs:subClassOf <http://www.co-ode.org/ontologies/pizza/pizza.owl#Food>
-> Explanation #1
	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.

	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#Cajun> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#OnionTopping>))
	 Axiom 2. ObjectPropertyDomain(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>)
-> Explanation #11
	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#Cajun> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperonataTopping>))
	 Axiom 2. ObjectPropertyDomain(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>)
-> Explanation #11
	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#Cajun> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/p

	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#American> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#MozzarellaTopping>))
	 Axiom 2. ObjectPropertyDomain(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>)
-> Explanation #17
	 Axiom 1. ObjectPropertyDomain(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>)
	 Axiom 2. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#American> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#TomatoTopping>))
-> Explanation #17
	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#American> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/on

In [19]:
explanations

'F:\\Documents\\VU\\KR\\kr_project2_explanation_by_forgetting\\datasets\\exp-pizza_super_simple-0'

In [20]:
exp_candidates = get_candidates_exp(explanations, candidate_objects)

In [21]:
exp_candidates

{'exp-1.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasIngredient': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping': 1},
 'exp-10.omn': {},
 'exp-11.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#TobascoPepperSauce': 1},
 'exp-12.omn': {},
 'exp-13.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#HamTopping': 1},
 'exp-14.omn': {},
 'exp-15.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasIngredient': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#TobascoPepperSauce': 1},
 'exp-16.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#OnionTopping': 1},
 'exp-17.omn': {'

In [22]:
func = 'min'
signature_path, ontology_path, sc_path = forget_from_exp(explanations, exp_candidates,func=func)

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-1.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 2, RBox: 1, ABox: 0 Signature: 5
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_signatures-pizza_super_simple-0\signature-exp-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@17853da

Input Number of Axioms: 3
Input Average Axiom size: 4.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping


Forgetting...
Started at Wed Oct 14 12:57:10 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting PeperoniSausageTopp |                   

Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-pizza_super_simple-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-15.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-pizza_super_simple-0\sc-exp-15.nt
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-16.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_signatures-pizza_super_simple-0\signature-exp-16.txt
Using forgetter uk.ac.man.cs.lethe.intern

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-pizza_super_simple-0\ont-exp-6.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-pizza_super_simple-0\ont-exp-6.owl
Format : RDF/XML Syntax
--------


Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-pizza_super_simple-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-6.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-pizza_super_simple-0\sc-exp-6.nt
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-7.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00

In [23]:
min_path = move_folders(signature_path, ontology_path, sc_path, func, path, input_ontology)

In [24]:
forgetting_loop(min_path, candidate_objects)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<?, ?it/s]


['http://www.co-ode.org/ontologies/pizza/pizza.owl#hasIngredient', 'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-1\ont-0.owl...
Simple: 0.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 5
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-1\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@297b3a

Input Number of Axioms: 2
Input Average Axiom size: 5.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasIngredient


Forgetting...
Started at Wed Oct 14 12:58:29 PKT 2020

 |                                                | 0/0 (NaN%) 

false


Finished at Wed Oct 14 12:58:29 PKT 2020
Duration: 48

Result Number of Axioms: 1
Result Average Axiom size: 5.0
R

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-1\ont-1.owl...
Simple: 0.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 1, TBox: 1, RBox: 0, ABox: 0 Signature: 3
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-1\signature-2.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@10f3d34

Input Number of Axioms: 1
Input Average Axiom size: 5.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping


Forgetting...
Started at Wed Oct 14 12:58:29 PKT 2020



...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<?, ?it/s]



Finished at Wed Oct 14 12:58:38 PKT 2020
Duration: 112

Result Number of Axioms: 3
Result Average Axiom size: 5.0
Result Definers: 0
Result Number Restrictions: 0

Exporting to result.owl...
F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-8 F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-8\signature-2.txt
--------------
[]
['http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping']


Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-9\ont-0.owl...
Simple: 0.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-9\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@297b3a

Input Number of Axioms: 2
Input Average Axiom size: 5.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping


Forgetting...
Started at Wed Oct 14 12:58:39 PKT 2020



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]

 |                                                | 0/1 (0.00%)                                                                  |                                                | 0/1 (0.00%) Forgetting hasTopping                                                                                      |                                                | 0/1 (0.00%) Forgetting hasTopping                                                                                      |================================================| 1/1 (100.00%

In [25]:
func = 'max'
signature_path, ontology_path, sc_path = forget_from_exp(explanations, exp_candidates,func=func)
max_path = move_folders(signature_path, ontology_path, sc_path, func, path, input_ontology)
forgetting_loop(max_path, candidate_objects)

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-1.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 2, RBox: 1, ABox: 0 Signature: 5
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_signatures-pizza_super_simple-0\signature-exp-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@17853da

Input Number of Axioms: 3
Input Average Axiom size: 4.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasIngredient


Forgetting...
Started at Wed Oct 14 13:46:54 PKT 2020

 |                                                | 0/0 (NaN%) 

false


Finished at Wed Oct 14 13:46:54 PKT 2020
Duration: 50

Result Number of Axioms: 1
Result Average Axiom size: 5.0


Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-16.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_signatures-pizza_super_simple-0\signature-exp-16.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@1c9af63

Input Number of Axioms: 2
Input Average Axiom size: 5.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping


Forgetting...
Started at Wed Oct 14 13:47:00 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting hasTopp |                                         

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-7.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 2, RBox: 1, ABox: 0 Signature: 5
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_signatures-pizza_super_simple-0\signature-exp-7.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@17853da

Input Number of Axioms: 3
Input Average Axiom size: 4.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping


Forgetting...
Started at Wed Oct 14 13:47:05 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting hasTopp |                                           

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 5008.72it/s]

Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_ont-pizza_super_simple-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-9.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_ont-pizza_super_simple-0\sc-exp-9.nt
['http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping', 'http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping']


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<?, ?it/s]



Finished at Wed Oct 14 13:47:13 PKT 2020
Duration: 120

Result Number of Axioms: 3
Result Average Axiom size: 5.666666666666667
Result Definers: 0
Result Number Restrictions: 0

Exporting to result.owl...
F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_pizza_super_simple\ont-exp-8 F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_pizza_super_simple\ont-exp-8\signature-1.txt
--------------
['http://www.co-ode.org/ontologies/pizza/pizza.owl#SauceTopping']


Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_pizza_super_simple\ont-exp-8\ont-1.owl...
Simple: 66.67%, EL: 66.67%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 6
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_pizza_super_simple\ont-exp-8\signature-2.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@7bddad

Input Number of Axioms: 3
Input Average Axiom size: 5.666666666666667
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#SauceTopping


Forgetting...
Started at Wed Oct 14 13:47:14 PKT 2020

 |                                                | 0/1 (0.00%)                                      

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


false


Finished at Wed Oct 14 13:47:14 PKT 2020
Duration: 124

Result Number of Axioms: 3
Result Average Axiom size: 5.0
Result Definers: 0
Result Number Restrictions: 0

Exporting to result.owl...
F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_pizza_super_simple\ont-exp-8 F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_pizza_super_simple\ont-exp-8\signature-2.txt
--------------
[]
[]


In [26]:
func = 'rand'
signature_path, ontology_path, sc_path = forget_from_exp(explanations, exp_candidates,func=func)
rand_path = move_folders(signature_path, ontology_path, sc_path, func, path, input_ontology)
forgetting_loop(rand_path, candidate_objects)

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-1.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 2, RBox: 1, ABox: 0 Signature: 5
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_signatures-pizza_super_simple-0\signature-exp-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@17853da

Input Number of Axioms: 3
Input Average Axiom size: 4.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping


Forgetting...
Started at Wed Oct 14 13:47:15 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting hasTopp |                                          

Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-pizza_super_simple-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-15.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-pizza_super_simple-0\sc-exp-15.nt
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-16.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_signatures-pizza_super_simple-0\signature-exp-16.txt
Using forgetter uk.ac.man.cs.lethe.in

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-pizza_super_simple-0\ont-exp-6.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-pizza_super_simple-0\ont-exp-6.owl
Format : RDF/XML Syntax
--------


Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-pizza_super_simple-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-6.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-pizza_super_simple-0\sc-exp-6.nt
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-7.omn...
Simple: 50.00%, EL: 100.00%, ALC: 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


['http://www.co-ode.org/ontologies/pizza/pizza.owl#hasIngredient']


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<?, ?it/s]



Finished at Wed Oct 14 13:47:36 PKT 2020
Duration: 122

Result Number of Axioms: 3
Result Average Axiom size: 5.0
Result Definers: 0
Result Number Restrictions: 0

Exporting to result.owl...
F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_pizza_super_simple\ont-exp-8 F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_pizza_super_simple\ont-exp-8\signature-2.txt
--------------
[]
['http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping']


Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_pizza_super_simple\ont-exp-9\ont-0.owl...
Simple: 0.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_pizza_super_simple\ont-exp-9\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@297b3a

Input Number of Axioms: 2
Input Average Axiom size: 5.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping


Forgetting...
Started at Wed Oct 14 13:47:36 PKT 2020



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3007.39it/s]

 |                                                | 0/1 (0.00%)                                                                  |                                                | 0/1 (0.00%) Forgetting hasTopping                                                                                      |                                                | 0/1 (0.00%) Forgetting hasTopping                                                                                      |================================================| 1/1 (100.00%

In [20]:
from difflib import SequenceMatcher

def similar(a, b):
    with open(a, 'r') as f:
        a = f.read()
    with open(b, 'r') as f:
        b = f.read() 
#     print((len(b.split(r'Object Properties'))))
#     print(len(b.splitlines()))
    if (len(a.splitlines())<= 12) or (len(b.splitlines()) <= 12):
        return(-1)
    
    return SequenceMatcher(None, a, b).ratio()

In [45]:
import statistics
def calc_avg_similarity(path, func,exclude = '_ont'):
    ontology_paths = [get_path([path,f ])for f in listdir(path) if 'ont' in f and exclude not in f]
    explanation_names = [f for f in listdir(path) if 'ont' in f and exclude not in f]
    result_dict = {}
    for _path,name in zip(ontology_paths, explanation_names):
        ontologies = [get_path([_path,f]) for f in listdir(_path) if isfile(get_path([_path,f])) and '.owl' in f]
        similarities = []
        print(f'Calculating similarities in {name}')
        if len(ontologies) < 2:
#             print('here')
            result_dict[f'{name}'] = -1
        for i,j in enumerate(range(1,len(ontologies))):
#             print(i,j)
            similarities.append(similar(ontologies[i],ontologies[j]))
        similarities = [i for i in similarities if i != -1]
        if not similarities:
#             print('here')
            result_dict[f'{name}'] = -1
        else:
            result_dict[f'{name}'] = statistics.mean(similarities)
#         print(similarities)
    return(result_dict)

In [46]:
max_dict = calc_avg_similarity(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_pizza_super_simple',
                   func ='max')

Calculating similarities in ont-exp-1
Calculating similarities in ont-exp-11
Calculating similarities in ont-exp-13
Calculating similarities in ont-exp-15
Calculating similarities in ont-exp-16
Calculating similarities in ont-exp-17
Calculating similarities in ont-exp-4
Calculating similarities in ont-exp-6
Calculating similarities in ont-exp-7
Calculating similarities in ont-exp-8
Calculating similarities in ont-exp-9


In [47]:
min_dict = calc_avg_similarity(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple',
                   func ='min')

Calculating similarities in ont-exp-1
Calculating similarities in ont-exp-11
Calculating similarities in ont-exp-13
Calculating similarities in ont-exp-15
Calculating similarities in ont-exp-16
Calculating similarities in ont-exp-17
Calculating similarities in ont-exp-4
Calculating similarities in ont-exp-6
Calculating similarities in ont-exp-7
Calculating similarities in ont-exp-8
Calculating similarities in ont-exp-9


In [48]:
rand_dict = calc_avg_similarity(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_pizza_super_simple',
                   func ='rand')

Calculating similarities in ont-exp-1
Calculating similarities in ont-exp-11
Calculating similarities in ont-exp-13
Calculating similarities in ont-exp-15
Calculating similarities in ont-exp-16
Calculating similarities in ont-exp-17
Calculating similarities in ont-exp-4
Calculating similarities in ont-exp-6
Calculating similarities in ont-exp-7
Calculating similarities in ont-exp-8
Calculating similarities in ont-exp-9


In [49]:
max_dict

{'ont-exp-1': -1,
 'ont-exp-11': -1,
 'ont-exp-13': -1,
 'ont-exp-15': -1,
 'ont-exp-16': -1,
 'ont-exp-17': -1,
 'ont-exp-4': -1,
 'ont-exp-6': -1,
 'ont-exp-7': -1,
 'ont-exp-8': 0.8806403688938687,
 'ont-exp-9': -1}

In [50]:
min_dict

{'ont-exp-1': 0.7745146839223495,
 'ont-exp-11': 0.6076854334226989,
 'ont-exp-13': 0.6098787341023366,
 'ont-exp-15': 0.7738392411382926,
 'ont-exp-16': 0.6469018677734955,
 'ont-exp-17': 0.6090828138913624,
 'ont-exp-4': 0.7712580805569369,
 'ont-exp-6': 0.7751861042183623,
 'ont-exp-7': 0.7742899850523169,
 'ont-exp-8': 0.8806403688938687,
 'ont-exp-9': 0.6086181277860326}

In [51]:
rand_dict

{'ont-exp-1': -1,
 'ont-exp-11': -1,
 'ont-exp-13': -1,
 'ont-exp-15': -1,
 'ont-exp-16': -1,
 'ont-exp-17': 0.6090828138913624,
 'ont-exp-4': -1,
 'ont-exp-6': 0.7751861042183623,
 'ont-exp-7': -1,
 'ont-exp-8': 0.8802175308927384,
 'ont-exp-9': 0.6086181277860326}

In [52]:
full_dict = dict(min_dict, **max_dict)
full_dict.update(rand_dict)

In [29]:
import pandas as pd

In [55]:
df =  pd.DataFrame([min_dict,max_dict,rand_dict]).T
df.columns = ['min','max','rand']
df.index.rename( 'ontology_generation', inplace=True)

In [57]:
df

,min,max,rand
ontology_generation,,,
ont-exp-1,0.774515,-1.00000,-1.000000
ont-exp-11,0.607685,-1.00000,-1.000000
ont-exp-13,0.609879,-1.00000,-1.000000
ont-exp-15,0.773839,-1.00000,-1.000000
ont-exp-16,0.646902,-1.00000,-1.000000
ont-exp-17,0.609083,-1.00000,0.609083
ont-exp-4,0.771258,-1.00000,-1.000000
ont-exp-6,0.775186,-1.00000,0.775186
ont-exp-7,0.774290,-1.00000,-1.000000


In [58]:
df.to_csv('pizza_super_simple_results.csv')

In [47]:
similar(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_pizza_super_simple\ont-exp-17\ont-0.owl',
       r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-17\ont-1.owl')

1.0

In [53]:
similar(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-1\ont-1.owl',
       r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-1\ont-0.owl')

0.7779990044798407

In [50]:
similar(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_pizza_super_simple\ont-exp-1\ont-0.owl',
       r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_pizza_super_simple\ont-exp-1\ont-1.owl')

0.9230310262529833

In [54]:
similar(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_HarryPotter_book\ont-exp-1\ont-0.owl',
       r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_HarryPotter_book\ont-exp-1\ont-1.owl')

0.5821850393700787

In [55]:
similar(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_HarryPotter_book\ont-exp-1\ont-0.owl',
       r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_HarryPotter_book\ont-exp-1\ont-1.owl')

0.5821850393700787